In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# cd /Users/allywang/Desktop/work/S2026/FACT

# python train_cutrun_cnn.py \
#   --fasta /path/to/mm10.fa \
#   --bigwig data/bws_for_predictor/SPT16_4D_2HdTAG_Sox2_S46_1_120_sorted.bw \
#   --chrom chr3 \
#   --window-size 10000 \
#   --stride 10000 \
#   --signal-bins 100 \
#   --batch-size 32 \
#   --epochs 10 \
#   --output-model cutrun_cnn.pt

In [ ]:
import sys 
sys.path.append('../src')

from cnn import * 
from datas import * 

In [ ]:
bigwig_path = '../data/bws_for_predictor/SPT16_4D_2HdTAG_SOX2_S46_1_120_sorted.bw'
fasta_path = '../data/GRCm38.primary_assembly.genome.fa'
chrom = 'chr3'
window_size = 2000
stride = 2000
signal_bins = None
batch_size = 256
epochs = 25
lr = 1e-3
device = 'mps'
weight_decay = 1e-4

In [7]:
dataset = SequenceBigWigDataset(
    fasta_path=str(fasta_path),
    bigwig_path=str(bigwig_path),
    signal_bins=signal_bins,
    chrom=chrom,
    window_size=window_size,
    stride=stride
)

In [8]:
len(dataset)

55660

In [9]:
x, y, y_count = next(iter(dataset))

x.shape, y.shape, y_count

(torch.Size([4, 2000]), torch.Size([2000]), tensor(262.7420))

In [10]:
config = TrainConfig(
    lr=lr,
    weight_decay=weight_decay,
    epochs=epochs,
    device=device
)

In [ ]:
model = train_cnn_regressor(
    dataset=dataset,
    batch_size=batch_size,
    config=config,
)

Epoch 1/25 - train loss: 9.0785, val loss: 7.8815


In [ ]:
import torch
import time
from pathlib import Path

date = time.strftime("%Y%m%d_%H")
output_model_path = Path(f'/Users/allywang/Desktop/work/S2026/models/seq_bpnet_{date}.pt')

torch.save(model.state_dict(), output_model_path)
print(f"Saved trained model state_dict to: {output_model_path}")

In [ ]:
%matplotlib inline

visualize_split_predictions(
    model,
    device='mps',
    n_examples_per_split=3,
)